# Week 6 - Characterizing egg variation
You read Egg Dimensions and Neonatal Mass of Shorebirds by Robert E. Ricklefs and want to see how the egg data we’ve been using in class compares to his results. Specifically, Ricklefs reported, “Coefficients of variation were 4 to 9% for egg volume” for shorebird eggs gathered in Manitoba, Canada. What is the range of coefficients of variation in our ASDN dataset?

The “coefficient of variation,” or CV, is a unitless measure of the variation of a sample, defined as the standard deviation divided by the mean and multiplied by 100 to express as a percentage. Thus, a CV of 10% means the standard deviation is 10% of the mean value. For the purposes of this computation, we will copy Ricklefs and use as a proxy for egg volume the formula

```W^2*L```

where 
 ```W``` is egg width and 
 ```L``` is egg length.

Your task is to create a Python program that reads data from the ASDN database and uses Pandas to compute, for each species in the database (for which there is egg data), the coefficient of variation of volume using the above formula. There are many ways this can be done. Because this assignment is primarily about programming in Python, please follow the steps below. Please submit your Python code when done.

In [67]:
import duckdb
import pandas as pd
import os

## Step 1
Create a query that will return the distinct species for which there is egg data (not all species and not all nests have egg data), so that you can then loop over those species. Your query should return two columns, species code and scientific name. Please order the results in alphabetic order of scientific name.

In [68]:
os.chdir("/Users/ims//MEDS/eds213-data/bren-eds213-data/database")
os.getcwd()

'/Users/ims/MEDS/eds213-data/bren-eds213-data/database'

In [70]:
conn = duckdb.connect('database.db')
c = conn.cursor()


In [71]:
distinct_egg_species = c.execute(
    '''
    SELECT DISTINCT Species.Code AS species_code, 
                    Species.Scientific_name AS "scientific name"
    FROM Bird_eggs 
    JOIN Bird_nests 
        ON Bird_eggs.Nest_ID = Bird_nests.Nest_ID
    JOIN Species 
        ON Bird_nests.Species = Species.Code
    ORDER BY Species.Scientific_name;           
    '''
    ).fetch_df()

In [72]:
distinct_egg_species

,species_code,scientific name
0,rutu,Arenaria interpres
1,dunl,Calidris alpina
2,wrsa,Calidris fuscicollis
3,sepl,Charadrius semipalmatus
4,reph,Phalaropus fulicarius
5,amgp,Pluvialis dominica
6,bbpl,Pluvialis squatarola


## Step 2
After you’ve connected to the database and created a csor c, iterate over the species like so:

```
species_query = """SELECT Code, Scientific_name FROM..."""
for row in c.execute(species_query).fetchall():  # DuckDB lame-o workaround
    species_code = row[0]
    scientific_name = row[1]
    # query egg data for that species (step 3)
    # compute statistics and print results (step 4)
```

In [74]:
species_query = """
    SELECT DISTINCT Species.Code AS species_code, 
                    Species.Scientific_name AS "scientific name"
    FROM Bird_eggs 
    JOIN Bird_nests 
        USING (Nest_ID)
    JOIN Species 
        ON Bird_nests.Species = Species.Code
    ORDER BY Species.Scientific_name;          
"""
    

    
for row in c.execute(species_query).fetchall():
    species_code = row[0]
    scientific_name = row[1]
    
    print(f"Species Code: {species_code}, Scientific Name: {scientific_name}")

Species Code: rutu, Scientific Name: Arenaria interpres
Species Code: dunl, Scientific Name: Calidris alpina
Species Code: wrsa, Scientific Name: Calidris fuscicollis
Species Code: sepl, Scientific Name: Charadrius semipalmatus
Species Code: reph, Scientific Name: Phalaropus fulicarius
Species Code: amgp, Scientific Name: Pluvialis dominica
Species Code: bbpl, Scientific Name: Pluvialis squatarola


## Step 3
You will need to construct a query that gathers egg data for a given species, one species at a time; the species code will be a parameter to that query. You can compute the formula

```W^2*L```

in SQL or in Pandas. For simplicity, SQL is suggested:

```egg_query = """SELECT Width*Width*Length AS Volume FROM..."""```

Within the loop, you will want to execute the query on the crent species in the loop iteration. You may use the Pandas function pd.read_sql to do so and also directly load the results into a dataframe:

```df = pd.read_sql(egg_query, conn, ...)```

Do a help(pd.read_sql) to figure out how to pass parameters to a query.

You may get a bunch of warnings from Pandas about how it “only supports SQLAlchemy…”. Just ignore them. (Sorry about that.)

In [75]:
    
for row in c.execute(species_query).fetchall():
    species_code = row[0]
    scientific_name = row[1]

    # Step 3: Query egg data for this species
    egg_data_query = """
        SELECT Bird_nests.Species, Egg_num, Length, Width
        FROM Bird_eggs
        JOIN Bird_nests USING (Nest_ID)
        WHERE Bird_nests.Species = ?
    """
    
    # Fetch egg data for this species
    df = c.execute(egg_data_query, [species_code]).fetch_df()
    
df

,Species,Egg_num,Length,Width
0,bbpl,1,55.560001,35.500000
1,bbpl,2,53.669998,35.900002
2,bbpl,3,53.509998,35.950001
3,bbpl,4,52.369999,36.000000
4,bbpl,1,55.450001,35.099998
5,bbpl,2,55.919998,36.029999
6,bbpl,3,53.400002,36.099998
7,bbpl,4,52.070000,36.349998
8,bbpl,1,51.459999,34.650002
9,bbpl,2,50.139999,34.830002


## Step 4
Finally, and still within your loop, you’ll want to compute statistics and print out the results:

```
cv = round(df.Volume.std()/df.Volume.mean()*100, 2)
print(f"{scientific_name} {cv}%")
```

Your output should look like this:

```
Arenaria interpres 21.12%
Calidris alpina 5.46%
Calidris fuscicollis 16.77%
Charadrius semipalmatus 8.99%
Phalaropus fulicarius 4.65%
Pluvialis dominica 19.88%
Pluvialis squatarola 6.94%
```

In [76]:
species_query = """
    SELECT DISTINCT Species.Code AS species_code, 
                    Species.Scientific_name AS "scientific name"
    FROM Bird_eggs 
    JOIN Bird_nests 
        USING (Nest_ID)
    JOIN Species 
        ON Bird_nests.Species = Species.Code
    ORDER BY Species.Scientific_name;          
"""
    
for row in c.execute(species_query).fetchall():
    species_code = row[0]
    scientific_name = row[1]

    # Step 3: Query egg data for this species
    egg_data_query = """
        SELECT Bird_nests.Species, Egg_num, Length, Width
        FROM Bird_eggs
        JOIN Bird_nests USING (Nest_ID)
        WHERE Bird_nests.Species = ?
    """
    
    # Fetch egg data for this species
    df = c.execute(egg_data_query, [species_code]).fetch_df()

    # Step 4: Calculate CV
    df['Volume'] = df['Width']**2 * df['Length']

    cv = round(df.Volume.std()/df.Volume.mean()*100, 2)
    print(f"{scientific_name} {cv}%")   


Arenaria interpres 21.12%
Calidris alpina 5.46%
Calidris fuscicollis 16.77%
Charadrius semipalmatus 8.99%
Phalaropus fulicarius 4.65%
Pluvialis dominica 19.88%
Pluvialis squatarola 6.94%


## Appendix

It’s not necessary to use pd.read_sql to get data into a dataframe, it’s just a convenience. To do so manually (and to show you it’s not that hard), imagine that your query returns three columns. Collect the row data into three separate lists, then manually create a dataframe specifying the contents as a dictionary:

```
rows = c.execute("SELECT Species, Width, Length FROM...").fetchall()
species_column = [t[0] for t in rows]
width_column = [t[1] for t in rows]
length_column = [t[2] for t in rows]

df = pd.DataFrame(
    {
        "species": species_column,
        "width": width_column,
        "length": length_column
    }
)
```